# imports

In [136]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from IPython.core.display import display, HTML
import re
import json
from nltk.tokenize import word_tokenize
import nltk
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity
import string
from nltk.corpus import names
from nltk.corpus import stopwords



from sklearn.feature_extraction.text import TfidfVectorizer


In [2]:
with open('movies_ive_seen_dict.json') as json_file: 
    movies = json.load(json_file) 

In [25]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [3]:
movies.keys()

dict_keys(['Joker', '127-Hours', 'Aliens', 'American-History-X', 'American-Psycho', 'Die-Hard', 'Godzilla', 'Mission-Impossible', 'Taxi-Driver', 'American-Pie', 'Big-Lebowski,-The', 'Bridesmaids', 'Crazy,-Stupid,-Love', 'Hangover,-The', 'Harold-and-Kumar-Go-to-White-Castle', 'Knocked-Up', 'Pineapple-Express', 'Superbad', 'Ted', 'Carrie', 'Final-Destination', 'Friday-the-13th', 'Get-Out', 'Halloween', 'Hannibal', 'It', 'Jaws', 'Pet-Sematary', 'Saw', 'Apocolyspe-Now', 'Braveheart', 'The-Patriot', 'The-Pianist', 'Platoon', 'Saving-Private-Ryan', 'Inglourious Basterds', 'Field-Of=Dreams', 'Finding-Nemo', 'Happy-Feet', 'Lion-King,-The', 'Marley-&-Me', 'Mulan', 'Pacifier,-The', 'Shrek', 'Up'])

# Processing

In [84]:
# work with single movie first

taxi_driver = movies['Taxi-Driver']

In [85]:
# processing  begins

text = str(movies['Taxi-Driver'])

In [86]:
# remove punctuation. DOES WORK!

text = re.sub('[%s]' % re.escape(string.punctuation),"", text)

In [87]:
# LOWER CASE EVERTHING

text = text.lower()

In [88]:
#remove numbers here

text = re.sub('\w*\d\w*', ' ', text)

In [140]:
names_all = ([name.lower() for name in names.words('male.txt')] +
             [name.lower() for name in names.words('female.txt')])

type(names_all)

list

In [133]:
# LEMMITIZE
lemmatizer = WordNetLemmatizer()
text = lemmatizer.lemmatize(text)

TypeError: unhashable type: 'list'

In [91]:
# REPLACE CHARACTERS I WANT TO REMOVE AND SPLIT ON

text = text.replace(' cut to', ' - ')
text = text.replace(' ext ', ' - ')
text = text.replace(' int ', ' - ')
text = text.replace(" cont ", ' - ')
text = text.replace(' b ', '')

In [92]:
text

'bb bbreakoutofframes script by website abstraction httpwsabstractcom over   free scripts here above notice must stay entact for useb if window top toplocationhreflocationhreftaxi driver by paul schrader property of the whole conviction of my life now rests upon the belief that loneliness far from being a rare and curious phenomenon is the central and inevitable fact of human existence thomas wolfe gods lonely man travis bickle age   lean hard the consummate loner on the surface he appears goodlooking even handsome he has a quiet steady look and a disarming smile which flashes from nowhere lighting up his whole face but behind that smile around his dark eyes in his gaunt cheeks one can see the ominous stains caused by a life of private fear emptiness and loneliness he seems to have wandered in from a land where it is always cold a country where the inhabitants seldom speak the head moves the expression changes but the eyes remain everfixed unblinking piercing empty space travis is now 

In [85]:
# pd.set_option('display.max_rows', None)
# tokens = word_tokenize(text)
# tokens

['bb',
 'bbreakoutofframes',
 'script',
 'by',
 'website',
 'abstraction',
 'httpwsabstractcom',
 'over',
 'free',
 'scripts',
 'here',
 'above',
 'notice',
 'must',
 'stay',
 'entact',
 'for',
 'useb',
 'if',
 'window',
 'top',
 'toplocationhreflocationhreftaxi',
 'driver',
 'by',
 'paul',
 'schrader',
 'property',
 'of',
 'the',
 'whole',
 'conviction',
 'of',
 'my',
 'life',
 'now',
 'rests',
 'upon',
 'the',
 'belief',
 'that',
 'loneliness',
 'far',
 'from',
 'being',
 'a',
 'rare',
 'and',
 'curious',
 'phenomenon',
 'is',
 'the',
 'central',
 'and',
 'inevitable',
 'fact',
 'of',
 'human',
 'existence',
 'thomas',
 'wolfe',
 'gods',
 'lonely',
 'man',
 'travis',
 'bickle',
 'age',
 'lean',
 'hard',
 'the',
 'consummate',
 'loner',
 'on',
 'the',
 'surface',
 'he',
 'appears',
 'goodlooking',
 'even',
 'handsome',
 'he',
 'has',
 'a',
 'quiet',
 'steady',
 'look',
 'and',
 'a',
 'disarming',
 'smile',
 'which',
 'flashes',
 'from',
 'nowhere',
 'lighting',
 'up',
 'his',
 'whole'

In [93]:
# SPLIT INTO DESIRED SECTIONS

text = text.split('-')
len(text)

42

In [94]:
text

['bb bbreakoutofframes script by website abstraction httpwsabstractcom over   free scripts here above notice must stay entact for useb if window top toplocationhreflocationhreftaxi driver by paul schrader property of the whole conviction of my life now rests upon the belief that loneliness far from being a rare and curious phenomenon is the central and inevitable fact of human existence thomas wolfe gods lonely man travis bickle age   lean hard the consummate loner on the surface he appears goodlooking even handsome he has a quiet steady look and a disarming smile which flashes from nowhere lighting up his whole face but behind that smile around his dark eyes in his gaunt cheeks one can see the ominous stains caused by a life of private fear emptiness and loneliness he seems to have wandered in from a land where it is always cold a country where the inhabitants seldom speak the head moves the expression changes but the eyes remain everfixed unblinking piercing empty space travis is now

# Vectorize - TF IDF

In [95]:
# set up stop words to be used in vectorizer

stops = stopwords.words('english') + names + ['b', 'black', 'scene', 'also']

# TF IDF
vectorizer = TfidfVectorizer(stop_words = stops)

# considered V matrix with NMF
doc_word = vectorizer.fit_transform(text)

/Users/samuelouimet/opt/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['luc', 'mar', 'zsa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [96]:
doc_word.shape

(42, 3620)

In [97]:
taxi_labels = ["Taxi-Driver" for i in text]

In [98]:
# show words as features

feature_names = vectorizer.get_feature_names()

feature_names

['abc',
 'able',
 'absorption',
 'abstraction',
 'accelerates',
 'accept',
 'accepts',
 'access',
 'accidents',
 'accomodate',
 'accompanied',
 'accompanying',
 'account',
 'acknowledge',
 'acme',
 'across',
 'acted',
 'action',
 'activate',
 'activated',
 'activates',
 'activity',
 'actors',
 'actresses',
 'actual',
 'actually',
 'add',
 'additional',
 'address',
 'admiring',
 'admiringly',
 'admit',
 'adulthood',
 'advancing',
 'advertising',
 'advisers',
 'affectionate',
 'affections',
 'afford',
 'afternoon',
 'againfor',
 'age',
 'aged',
 'agents',
 'ages',
 'aggressive',
 'ago',
 'agony',
 'agreed',
 'ah',
 'ahead',
 'ahh',
 'aide',
 'aides',
 'aim',
 'aint',
 'air',
 'airplanethats',
 'airport',
 'aisle',
 'ajar',
 'album',
 'alert',
 'alive',
 'alkaseltzer',
 'allamerica',
 'allamerican',
 'allnight',
 'almost',
 'alone',
 'along',
 'already',
 'alright',
 'altered',
 'alternate',
 'although',
 'always',
 'amazed',
 'amazement',
 'americas',
 'amiable',
 'among',
 'amsterdam',


# NMF

In [117]:
# NMF MODEL
# H matrix 

nmf_model = NMF(6)
doc_topic = nmf_model.fit_transform(doc_word)

In [118]:
# W matrix in NMF

topic_word = pd.DataFrame(nmf_model.components_.round(3),
             index = ["component_1","component_2",
                     "component_3","component_4",
                     "component_5", "component_6"]
                          ,columns = vectorizer.get_feature_names())
topic_word

,abc,able,absorption,abstraction,accelerates,accept,accepts,access,accidents,accomodate,...,younger,youre,youth,youve,zap,zip,zips,zodys,zooming,zoomingin
component_1,0.008,0.010,0.000,0.000,0.009,0.013,0.014,0.000,0.000,0.001,...,0.000,0.084,0.008,0.003,0.010,0.000,0.011,0.001,0.010,0.005
component_2,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.007,...,0.011,0.034,0.000,0.015,0.001,0.000,0.001,0.007,0.001,0.000
component_3,0.008,0.005,0.000,0.000,0.000,0.001,0.001,0.000,0.000,0.000,...,0.000,0.036,0.008,0.028,0.000,0.000,0.000,0.000,0.000,0.001
component_4,0.000,0.000,0.014,0.000,0.000,0.000,0.000,0.000,0.018,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.009
component_5,0.000,0.000,0.000,0.001,0.000,0.011,0.000,0.021,0.000,0.001,...,0.000,0.000,0.000,0.000,0.000,0.014,0.000,0.001,0.000,0.001
component_6,0.000,0.000,0.000,0.016,0.000,0.004,0.004,0.000,0.004,0.002,...,0.000,0.009,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.002


In [119]:
# show the created topics and some words in them 

display_topics(nmf_model, vectorizer.get_feature_names(), 20)


Topic  0
sport, dont, man, old, get, go, like, know, want, im, people, gun, well, one, thats, walks, right, looks, got, think

Topic  1
wizard, doughboy, killer, know, says, yeah, hey, street, melio, pause, back, got, around, line, look, voice, thats, hello, cab, dont

Topic  2
palantine, headquarters, assistant, traffic, taxi, passes, hotel, sir, day, chicken, regis, st, aides, aide, ten, another, back, front, cab, mass

Topic  3
garage, passenger, taxi, pulls, cab, clean, young, driveway, seat, cabbie, nights, return, jacket, blood, duty, night, professional, vocontd, pocket, back

Topic  4
secret, service, crowd, man, platform, palantine, rally, standing, jacket, applause, taxi, side, look, looks, moving, policeman, scanning, cameras, face, eyes

Topic  5
table, letter, wall, vo, life, mattress, room, apartment, voice, couples, headline, plain, male, read, shot, cannot, longer, new, slow, motion


In [120]:
# H matrix. Shows the docs we started with, and topics in them

doc_topic_nmf = pd.DataFrame(doc_topic.round(5),
                             index = taxi_labels,
                             columns = ["component_1","component_2",
                     "component_3","component_4","component_5","component_6" ])
doc_topic_nmf

,component_1,component_2,component_3,component_4,component_5,component_6
Taxi-Driver,0.00104,0.00869,0.00000,0.00000,0.03025,0.32619
Taxi-Driver,0.00000,0.00000,0.00000,0.42449,0.00000,0.00000
Taxi-Driver,0.11712,0.00000,0.00923,0.23210,0.10155,0.02262
Taxi-Driver,0.00000,0.00000,0.00000,0.00000,0.00000,0.43609
Taxi-Driver,0.00000,0.00000,0.00000,0.58475,0.00000,0.00000
Taxi-Driver,0.00974,0.00000,0.00000,0.43281,0.00000,0.13286
Taxi-Driver,0.11559,0.00000,0.00000,0.06495,0.00000,0.00291
Taxi-Driver,0.00000,0.00518,0.00000,0.02854,0.00000,0.26780
Taxi-Driver,0.18956,0.00007,0.27191,0.00000,0.00223,0.04324
Taxi-Driver,0.00000,0.00000,0.33115,0.12375,0.00000,0.00000


# Notes

# CREATE PROCESSING FUNCTION

In [184]:
# work with single movie first

taxi_driver2 = movies['Taxi-Driver']

In [281]:
def clean_split(script):

    # covert script to string
    script_ = str(script)

    # remove punctuation
    script_ = re.sub('[%s]' % re.escape(string.punctuation),"", script_)

    # lower case everything
    script_ = script_.lower()

    # remove numbers here
    script_ = re.sub('\w*\d\w*', ' ', script_)
    
    # replace characters I want to split on
    script_ = script_.replace(' cut to', ' - ')
    script_ = script_.replace(' ext ', ' - ')
    script_ = script_.replace(' int ', ' - ')
    script_ = script_.replace(" cont ", ' - ')
    script_ = script_.replace(' b ', '')

    # split into desired sections
    script_ = script_.split('-')
    
    return script_

In [365]:
#custome stops
movie_term_stops = ['director', 'producer','camera','angle',
                    'os','oc','character','parenthetical',
                    'slug','line','script']
others = ['b', 'black', 'scene','also',
          'joker','melio','cammeron','sweeney', 
          'rasmussen','im','get','got', 'na','gon','vo', 'ta', 'oh','oz']

names_plural = [i+'s' for i in names_all]


In [359]:
def tokenize(split_script):
    
    text_ = split_script
    
    # lemmatization 
    lemmatizer = WordNetLemmatizer()
    text_ = lemmatizer.lemmatize(text_)
      
    # tokenize
    tokens = word_tokenize(text_)
    
    # stop words
    stops_apost = stopwords.words('english') + names_all + movie_term_stops + others + names_plural
    stops = [i.replace("'", '') for i in stops_apost]
    tokens_stop = [i for i in tokens if i not in stops]
    
    return tokens_stop

# NMF Multiple Scripts

In [368]:
#split 2 moview and combine. Need to automate this somehow.
taxi_after_split = clean_split(taxi_driver2)
joker_after_split = clean_split(movies['Joker'])
american_after_split = clean_split(movies['American-History-X'])
am_pie_split = clean_split(movies['American-Pie'])

multiple_movies = taxi_after_split + joker_after_split + american_after_split + am_pie_split


#create labels for groupyby later
taxi_labels = ["Taxi-Driver" for i in taxi_after_split]
joker_labels = ["Joker" for i in joker_after_split]
american_labels = ['American-History-X' for i in american_after_split]
am_pie_labels = ['American-Pie' for i in am_pie_split]
movie_labels = taxi_labels + joker_labels + american_labels + am_pie_labels

len(movie_labels)

524

In [369]:
# Customize TF-IDF function
vectorizer1 = TfidfVectorizer(tokenizer=tokenize, min_df=3)

# Apply it to my docs
doc_word1 = vectorizer1.fit_transform(multiple_movies)

In [370]:
# NMF MODEL
# H matrix 

nmf_model1 = NMF(10)
doc_topic1 = nmf_model1.fit_transform(doc_word1)



In [371]:
# W matrix in NMF

topic_word = pd.DataFrame(nmf_model1.components_.round(3),
             index = ["component_1","component_2",
                     "component_3","component_4",
                     "component_5","component_6","component_7",
                     "component_8","component_9",
                     "component_10"]
                          ,columns = vectorizer1.get_feature_names())
topic_word

,aback,able,acknowledge,across,act,action,actually,address,adjacent,adopted,...,yellow,yells,yep,yes,yet,york,young,younger,youth,zips
component_1,0.033,0.037,0.014,0.019,0.016,0.035,0.047,0.004,0.005,0.012,...,0.000,0.025,0.021,0.112,0.012,0.012,0.119,0.021,0.019,0.017
component_2,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.038,0.000,...,0.021,0.000,0.012,0.108,0.030,0.000,0.000,0.005,0.000,0.000
component_3,0.000,0.001,0.000,0.000,0.003,0.000,0.005,0.014,0.000,0.000,...,0.002,0.000,0.000,0.030,0.022,0.000,0.000,0.000,0.000,0.000
component_4,0.000,0.007,0.000,0.026,0.000,0.000,0.021,0.024,0.000,0.000,...,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.036,0.000,0.000
component_5,0.000,0.003,0.001,0.072,0.000,0.000,0.000,0.000,0.000,0.002,...,0.000,0.031,0.016,0.017,0.000,0.000,0.000,0.000,0.000,0.000
component_6,0.000,0.000,0.003,0.184,0.000,0.010,0.000,0.009,0.003,0.000,...,0.066,0.000,0.000,0.000,0.007,0.017,0.122,0.000,0.004,0.000
component_7,0.006,0.004,0.000,0.011,0.049,0.008,0.000,0.000,0.004,0.040,...,0.000,0.006,0.000,0.007,0.015,0.000,0.017,0.000,0.000,0.003
component_8,0.007,0.000,0.000,0.000,0.000,0.000,0.021,0.000,0.000,0.000,...,0.000,0.000,0.013,0.000,0.000,0.000,0.000,0.000,0.000,0.000
component_9,0.001,0.000,0.000,0.010,0.000,0.000,0.000,0.000,0.000,0.014,...,0.000,0.000,0.000,0.000,0.009,0.000,0.000,0.000,0.000,0.001
component_10,0.000,0.000,0.000,0.000,0.000,0.004,0.001,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.052,0.000,0.017,0.000,0.002,0.000


In [373]:
# show the created topics and some words in them 

display_topics(nmf_model1, vectorizer1.get_feature_names(), 20)


Topic  0
know, man, like, looks, fucking, yeah, want, back, go, think, right, beat, thats, walks, shit, okay, well, come, fuckin, cant

Topic  1
bedroom, day, computer, shit, holy, another, phone, time, screen, hes, going, pillow, yes, reads, brother, types, bed, go, cheer, somebody

Topic  2
falls, east, great, high, day, bathroom, courtyard, stall, sits, school, senior, lockers, library, talking, alone, stifler, main, different, like, steps

Topic  3
stifler, stiflers, house, brothers, brother, cottage, beer, night, door, party, chick, fuck, happened, mom, continuing, room, hell, kitchen, bedroom, michigan

Topic  4
dad, house, continuing, door, hallway, day, stairs, mom, downstairs, stairwell, carrying, parents, approaches, crashes, gets, sprints, hey, lawn, across, doorknob

Topic  5
street, taxi, palantine, car, cab, toward, across, front, man, side, back, building, wizard, face, running, watches, pulls, window, two, crowd

Topic  6
mom, door, tv, room, apartment, close, jokers, 

In [374]:
# H matrix. Shows the docs we started with, and topics in them

doc_topic_nmf1 = pd.DataFrame(doc_topic1.round(5),
                             index = movie_labels,
                             columns = ["component_1","component_2",
                     "component_3","component_4","component_5","component_6","component_7",
                     "component_8","component_9",
                     "component_10"])
doc_topic_nmf1

,component_1,component_2,component_3,component_4,component_5,component_6,component_7,component_8,component_9,component_10
Taxi-Driver,0.03724,0.00000,0.00000,0.00000,0.00000,0.07058,0.03995,0.00700,0.00000,0.0
Taxi-Driver,0.00000,0.00000,0.00277,0.00000,0.00000,0.12443,0.00000,0.00000,0.00004,0.0
Taxi-Driver,0.05115,0.00192,0.00027,0.00000,0.00000,0.23003,0.00000,0.00263,0.00413,0.0
Taxi-Driver,0.01504,0.00000,0.00000,0.00000,0.00000,0.04771,0.08868,0.00148,0.00000,0.0
Taxi-Driver,0.00000,0.00000,0.00000,0.00000,0.00000,0.17648,0.00000,0.00000,0.00000,0.0
...,...,...,...,...,...,...,...,...,...,...
American-Pie,0.14367,0.00000,0.04440,0.00477,0.00000,0.00000,0.00000,0.06876,0.00000,0.0
American-Pie,0.03781,0.17581,0.00000,0.02072,0.00000,0.00000,0.00000,0.00000,0.00444,0.0
American-Pie,0.03948,0.00532,0.15657,0.01742,0.00000,0.01597,0.01614,0.00000,0.00000,0.0
American-Pie,0.02299,0.04512,0.04096,0.00000,0.35297,0.00000,0.00000,0.00000,0.00000,0.0


In [375]:
doc_topic_nmf1.groupby(level=0).sum()

,component_1,component_2,component_3,component_4,component_5,component_6,component_7,component_8,component_9,component_10
American-History-X,7.76733,2.68261,0.89756,0.93503,1.88679,5.22303,4.17283,0.27470,0.40865,0.80272
American-Pie,5.90421,10.23287,12.39292,10.58753,8.12820,1.48637,2.45000,8.33368,4.92859,6.81547
Joker,3.31752,0.61728,0.75505,0.13318,0.51651,6.66221,14.75323,0.26412,0.84780,0.53150
Taxi-Driver,1.97548,0.15369,0.25549,0.01322,0.01735,6.01505,0.87757,0.13404,0.20207,0.49369


# Notes

In [121]:
# NEED TO IMPROVE MODEL

# do i need to normalize here before aggregation?
# TRY ONLY ADJECTIVES AND VERBS
# LDA
# SPLIT MORE?